In this code, we explain how to filter out data, select appropriate variables, and perform feature engineering.
The steps are:
1. Import libraries and read three datasets (liver_df, wl_data_df, followup_df)
2. Select appropriate variables from liver_df
3. Filter out patients from liver_df
4. Perform a feature engineering from wl_data_df, and merge with liver_df
5. Encode certain variables from liver_df
6. Merge the liver_df with followup_df
7. Identify the outcome, store the final dataframe into a new csv file


In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

liver_df = pd.read_stata("/content/drive/MyDrive/Borealis AI/LIVER_DATA.DTA")
wl_data_df = pd.read_stata("/content/drive/MyDrive/Borealis AI/LIVER_WLHISTORY_DATA.DTA")
followup_df = pd.read_stata("/content/drive/MyDrive/Borealis AI/Dataset/LIVER_FOLLOWUP_DATA.DTA")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-17cafd6a06fb>:7: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  wl_data_df = pd.read_stata("/content/drive/MyDrive/Borealis AI/LIVER_WLHISTORY_DATA.DTA")


1. In the above code, three datasets were stored into liver_df, wl_data_df and followup_df.
- liver_df: baseline patient information, containing either socioeconiomic features (gender, ethnicity), or the initial and final clinical variables (bilirubin level at the registration of the liver transplant, and the bilirubin level right before the liver transplant)
- wl_data_df: contains all the clinical values from the waiting list (between the initial and fianl value)
- followup_df: contains post-transplant medical check-up values

2. We start by selecting necessary features from the liver_df. Features were selected based on the risk adjustment models from Sceintific Registry of Transplant Recipients (SRTR).


In [2]:
liver_columns = ['AGE', 'AGE_GROUP', 'ABO', 'BACT_PERIT_TCR', 'BMI_CALC', 'DIAB', 'ETHCAT', 'ETHCAT_DON', 'GENDER',
           'HGT_CM_CALC', 'LIFE_SUP_TCR', 'MALIG_TCR', 'HCC_DIAGNOSIS_TCR', 'PREV_AB_SURG_TCR',
           'TIPSS_TCR', 'INIT_WGT_KG', 'WORK_INCOME_TCR', 'INIT_ALBUMIN',
           'FINAL_ALBUMIN', 'INIT_BILIRUBIN', 'FINAL_BILIRUBIN',
           'INIT_DIALYSIS_PRIOR_WEEK', 'FINAL_DIALYSIS_PRIOR_WEEK', 'INIT_ENCEPH',
           'FINAL_ENCEPH', 'INIT_INR', 'FINAL_INR', 'INIT_SERUM_CREAT',
           'FINAL_SERUM_CREAT', 'INIT_SERUM_SODIUM', 'FINAL_SERUM_SODIUM', 'DGN_TCR',
           'INIT_MELD_PELD_LAB_SCORE', 'FINAL_MELD_PELD_LAB_SCORE', 'MELD_PELD_LAB_SCORE', 'DAYSWAIT_CHRON',
           'GSTATUS', 'GTIME', 'MULTIORG', 'PREV_TX_ANY', 'TX_YEAR', 'REM_CD', 'INIT_MELD_OR_PELD', 'FINAL_MELD_OR_PELD',
           'WL_ID_CODE', 'INIT_DATE', 'TX_DATE', 'PT_CODE', 'TRR_ID_CODE']
filtered_df = liver_df[liver_columns]
filtered_df = filtered_df.replace('', np.nan)

3. We then filter out patietns based on specific criteria:
- Patients received transplantation between June 18th, 2018 and June 18th, 2023 (arbitrarily chosen time period)
- Adult patients
- Patients who received deceased organs
- Patients not registered for multi-organ transplantation
- Patients with the history of previous transplantation
- Patients who used MELD system


In [3]:
filtered_df = filtered_df[filtered_df['INIT_DATE'] >= pd.Timestamp('2018-06-18')]
filtered_df = filtered_df[filtered_df['INIT_DATE'] <= pd.Timestamp('2023-06-18')]
filtered_df = filtered_df[filtered_df['AGE_GROUP'] == 'A']
filtered_df = filtered_df[filtered_df['MULTIORG'] != 'Y']
filtered_df = filtered_df[filtered_df['PREV_TX_ANY'] == 'N']
filtered_df = filtered_df[filtered_df['REM_CD'] == 4]
filtered_df = filtered_df[filtered_df['INIT_MELD_OR_PELD'] == 'MELD']
filtered_df = filtered_df[filtered_df['FINAL_MELD_OR_PELD'] == 'MELD']
filtered_df.shape

(33120, 49)

4. Perform a feature engineering in waiting list data
- We will first only select patients that are in the final cohort (filtered_df)
- wl_data_df and filtered_df is connected by WL_ID_CODE, which is a unique patient identifier for waiting list history
- We will calculate the linear slope of clinical data of five variables: bilirubin, albumin, creatinine, sodium and INR

In [4]:
import datetime

start_date = '2018-06-18'
end_date = '2023-06-18'

filt_wl_df = wl_data_df[(wl_data_df['CHG_DATE'] >= start_date) & (wl_data_df['CHG_DATE'] <= end_date)]
grouped = filt_wl_df.groupby('WL_ID_CODE')

def calculate_slope(group, column_name):
    if len(group) == 1:
        return 0
    else:
        x = np.arange(len(group))
        y = group[column_name].values
        slope, _ = np.polyfit(x, y, 1)
        return slope

columns_to_calculate = ['BILIRUBIN', 'ALBUMIN', 'SERUM_SODIUM', 'INR', 'SERUM_CREAT']

slopes_df = pd.DataFrame()

for column in columns_to_calculate:
    slopes = grouped.apply(calculate_slope, column_name=column).reset_index(name=f'{column}_SLOPE')
    slopes_df = pd.concat([slopes_df, slopes[f'{column}_SLOPE']], axis=1)

slopes_df['WL_ID_CODE'] = grouped.groups.keys()

final_df = pd.merge(slopes_df, filtered_df, on = 'WL_ID_CODE')

5. Now we encode certian variables from liver_df. Before we do that, we will drop some columns that are no longer necessary

In [5]:
final_df.drop(['WL_ID_CODE', 'AGE_GROUP', 'MULTIORG', 'PREV_TX_ANY', 'TX_YEAR', 'REM_CD', 'INIT_MELD_OR_PELD', 'INIT_DATE'], axis = 1, inplace = True)
final_df.columns

Index(['BILIRUBIN_SLOPE', 'ALBUMIN_SLOPE', 'SERUM_SODIUM_SLOPE', 'INR_SLOPE',
       'SERUM_CREAT_SLOPE', 'AGE', 'ABO', 'BACT_PERIT_TCR', 'BMI_CALC', 'DIAB',
       'ETHCAT', 'ETHCAT_DON', 'GENDER', 'HGT_CM_CALC', 'LIFE_SUP_TCR',
       'MALIG_TCR', 'HCC_DIAGNOSIS_TCR', 'PREV_AB_SURG_TCR', 'TIPSS_TCR',
       'INIT_WGT_KG', 'WORK_INCOME_TCR', 'INIT_ALBUMIN', 'FINAL_ALBUMIN',
       'INIT_BILIRUBIN', 'FINAL_BILIRUBIN', 'INIT_DIALYSIS_PRIOR_WEEK',
       'FINAL_DIALYSIS_PRIOR_WEEK', 'INIT_ENCEPH', 'FINAL_ENCEPH', 'INIT_INR',
       'FINAL_INR', 'INIT_SERUM_CREAT', 'FINAL_SERUM_CREAT',
       'INIT_SERUM_SODIUM', 'FINAL_SERUM_SODIUM', 'DGN_TCR',
       'INIT_MELD_PELD_LAB_SCORE', 'FINAL_MELD_PELD_LAB_SCORE',
       'MELD_PELD_LAB_SCORE', 'DAYSWAIT_CHRON', 'GSTATUS', 'GTIME',
       'FINAL_MELD_OR_PELD', 'TX_DATE', 'PT_CODE', 'TRR_ID_CODE'],
      dtype='object')

In [6]:
# encoding values for diabetes
diab_map = {
    1: 'N',
    2: 'Type1',
    3: 'Type2',
    4: 'Other',
    5: 'Other',
    998: 'Other'
}
final_df['DIAB'] = final_df['DIAB'].map(diab_map)

# encoding values for ethnicity
eth_map = {
    1: 'White',
    2: 'Black',
    4: 'Hispanic',
    5: 'Asian',
    6: 'Native',
    9: 'Multiracial'
}
final_df['ETHCAT'] = final_df['ETHCAT'].map(eth_map)
final_df['ETHCAT_DON'] = final_df['ETHCAT_DON'].map(eth_map)

# encoding values for blood type:
blood_map = {
    'A1': 'A',
    'A1B': 'AB',
    'A2': 'A',
    'A2B': 'AB',
    'A': 'A',
    'AB': 'AB',
    'O': 'O',
    'B': 'B'
}
final_df['ABO'] = final_df['ABO'].map(blood_map)

6. Merge the followup data with the original data. TRR_ID_CODE is a unique patient identifier between liver_df and followup_df

In [7]:
fdf = pd.merge(followup_df, final_df[['TRR_ID_CODE', 'TX_DATE']], on = 'TRR_ID_CODE', how = 'left')
fdf = fdf[(fdf['TRR_ID_CODE']).isin(final_df['TRR_ID_CODE'].values)]
fdf[['TRR_ID_CODE', 'TX_DATE', 'PX_STAT_DATE', 'GRF_FAIL_DATE']]

,TRR_ID_CODE,TX_DATE,PX_STAT_DATE,GRF_FAIL_DATE
1487699,A774971,2018-06-21,2018-12-24,NaT
1487700,A774971,2018-06-21,2019-06-20,NaT
1487701,A774971,2018-06-21,2020-06-24,NaT
1487702,A774971,2018-06-21,2020-06-24,NaT
1487703,A774971,2018-06-21,2021-07-01,NaT
...,...,...,...,...
1674435,A984208,2023-07-24,2023-08-21,NaT
1674599,A984814,2023-07-28,2023-09-17,NaT
1674685,A985443,2023-07-29,2023-07-29,NaT
1675150,A987079,2023-08-13,2023-09-30,NaT


In [8]:
def filter_dates_180(row):
    within_180_days = row['PX_STAT_DATE'] <= (row['TX_DATE'] + pd.Timedelta(days=180))
    before_grf_fail = pd.isnull(row['GRF_FAIL_DATE']) or row['PX_STAT_DATE'] < row['GRF_FAIL_DATE']
    return within_180_days and before_grf_fail

fdf_180 = fdf[fdf.apply(filter_dates_180, axis = 1)]
fdf_180 = fdf_180.sort_values(by = ['TRR_ID_CODE', 'PX_STAT_DATE'])
fdf_180 = fdf_180.drop_duplicates(subset = ['TRR_ID_CODE'], keep = 'last')
# PX_STAT_DATE is a date of medical checkup, so within_180_days check if there is any medical checkup value within 180 days of transplantation
# before_grf_fail makes sure that we are only getting a medical check-up data that is BEFORE the graft failre date

In [9]:
fdf_180 = fdf_180[['BMI', 'CREAT', 'HBSAG', 'HBV_CORE', 'HBV_DNA', 'HCV_NAT', 'HCV_SEROLOGY', 'HIV_NAT', 'HIV_SEROLOGY', 'TBILI', 'TRR_ID_CODE']]
final_180 = pd.merge(final_df, fdf_180, on = 'TRR_ID_CODE', how = 'left')

7. Identify the outcome, store the dataset into a separate csv file

In [10]:
final_180['180_OUTCOME'] = np.where((final_180['GSTATUS'] == 1) & (final_180['GTIME'] <= 180), 1, 0)
final_180.drop(['GSTATUS', 'GTIME'], axis = 1)
final_180.to_csv("final_180.csv")